In [17]:
import sys
# sys.path.append('/opt/conda/bin/python')
print(sys.executable)

/opt/conda/bin/python


In [18]:
import numpy as np
import skimage.io as io
import random
import os
import albumentations as A
import cv2
import pickle
import torch
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from torchvision import transforms


### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

In [19]:
torch.cuda.device_count()
device = torch.device('cuda:5')

In [20]:
torch.cuda.empty_cache()

In [21]:
mode = 'train'

load_folder = '/shared/data/COCOdataset2017'
save_folder = '/shared/suhyeon/GCCdataset'
mask_folder = '{}/masks/{}'.format(save_folder, mode)

batch_size = 1
input_size = (256, 256)
top5_dic = defaultdict(tuple)
SMOOTH = 1e-6

In [22]:
with open('./validAnns_train.pkl', 'rb') as f:
    validAnns = pickle.load(f)
    
with open('./validAnns_dict_train.pkl', 'rb') as f:
    validAnnsDict = pickle.load(f)

In [23]:
trans = transforms.ToTensor()

In [24]:
import math
import time

masks = {}

start = time.time()
for i, (catId, imgId, annId) in enumerate(validAnns):
    mask = cv2.imread(mask_folder + '/' + str(annId) + '.png', cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, input_size)
    mask = trans(mask).type(torch.bool).to(device)
    masks[annId] = mask
end = time.time()

print(f"data loading: {end - start:.5f} sec")

data loading: 304.71742 sec


In [ ]:
total = 0
src_validAnns = validAnns[100000:120000]

for i, (catId_1, imgId_1, annId_1) in enumerate(src_validAnns):
    start = time.time()
    ious = [(0, 0, 0)] * 5
    mask_1 = masks[annId_1]
    
    cands = [data for data in validAnns if data[0]==catId_1 and data[2] != annId_1]
    
    for catId_2, imgId_2, annId_2 in cands:
        mask_2 = masks[annId_2]

        intersection = (mask_1 & mask_2).sum()
        union = (mask_1 | mask_2).sum()
        iou = (intersection + SMOOTH) / (union + SMOOTH)  # to avoid 0/0
        if ious[-1][2] < iou:
            ious[-1] = (imgId_2, annId_2, iou)
            ious = sorted(ious, key=lambda x : -x[2])

    end = time.time()
    elapsed = end - start
    total += elapsed
    print("[{}/{}]".format(i, len(src_validAnns)), f"average time: {total/(i+SMOOTH):.5f} sec", end='\r')
    top5_dic[(catId_1, imgId_1, annId_1)] = ious
    
    if i % 100 == 0:
        with open("./top5_dict_train_100000.pkl".format(mode), "wb") as f:
            pickle.dump(top5_dic, f)
with open("./top5_dict_train_100000.pkl".format(mode), "wb") as f:
            pickle.dump(top5_dic, f)

In [ ]:
with open("./top5_dict_train_80000.pkl".format(mode), "wb") as f:
            pickle.dump(top5_dic, f)